In [7]:
import numpy as np
import matplotlib.pyplot as plt

# Task 7
Question 7: Set up a numerical scheme to solve (6) and use it to solve the
equation with injection initial data (again do not hesitate to consider special
cases).

$
(\phi_s)_t + (f(s)u)_x - (g(s)s_x)_x = 0
$


In [3]:
# defining constants

phi = 0.2
k = 1.0
mu_g = 1.0
mu_w = 1.0
P0 = 1.0
u = 1.0

phi_derivative = P0



In [ ]:
# defining mobility functions

def lambda_g(s):
    return s / mu_g

def lambda_w(s):
    return (1 - s) / mu_w

# defining f and g
def f(s):
    return lambda_g(s) / (lambda_w(s) + lambda_g(s))

def g(s):
    frac = lambda_w(s)*lambda_g(s) / (lambda_w(s) + lambda_g(s))
    return frac * k * phi_derivative


In [ ]:
# define the grid and time step

L = 1.0
N = 100
dx = L / N
x = np.linspace(0, L, N)
dt = 0.01
t = np.arange(0, 1, dt)

# initial condition: injection at x=0

s = np.zeros(N)
s_left = 1.0  # injection saturation
s[0] = 0

F = np.zeros(N)

# first for f(s) term using upwind scheme

F[0] = f(s_left) 

if u > 0:
    F[1:N] = f(s[:-1]) * u
else:
    F[1:N] = f(s[1:]) * u


# time-stepping loop
